In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [3]:
sc = SparkContext()
ssc = StreamingContext(sc, 10)
sqlContext = SQLContext(sc)

23/09/17 17:06:26 WARN Utils: Your hostname, juan-MS-7A57 resolves to a loopback address: 127.0.1.1; using 192.168.20.22 instead (on interface enp6s0)
23/09/17 17:06:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/17 17:06:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/17 17:06:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
/home/juan/2023/Spark-Tutorials/.venv/lib/python3.10/site-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(
/home/juan/2023/Spark-Tutorials/.venv/lib/python3.10/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  wa

In [4]:
socket_stream = ssc.socketTextStream('127.0.0.1', 9000)

In [5]:
lines = socket_stream.window(20)

In [6]:
from collections import namedtuple

In [7]:
fields = ('tag', 'count')

In [8]:
Tweet = namedtuple('Tweet', fields)

In [9]:
(lines.flatMap(lambda text: text.split(' '))
    .filter(lambda word: word.lower().startswith('#'))
    .map(lambda word: (word.lower(), 1))
    .reduceByKey(lambda a, b: a + b)
    .map(lambda rec: Tweet(rec[0], rec[1]))
    .foreachRDD(lambda rdd: rdd.toDF().sort(desc('count'))
    .limit(10).registerTemplate('tweets')))

In [10]:
ssc.start()

In [11]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
count = 0
while count < 10:
    time.sleep(3)
    top_10_tweets = sqlContext.sql('Select tag, count from tweets')
    top_10_df = top_10_tweets.toPandas()
    display.clear_output(wait=True)
    sns.plt.figure(figsize=(10,8))
    sns.barplot(x='count', y='tag', data=top_10_df)
    sns.plt.show()
    count += 1

23/09/17 17:06:31 ERROR JobScheduler: Error running job streaming job 1694988390000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/home/juan/2023/Spark-Tutorials/.venv/lib/python3.10/site-packages/pyspark/streaming/util.py", line 71, in call
    r = self.func(t, *rdds)
  File "/home/juan/2023/Spark-Tutorials/.venv/lib/python3.10/site-packages/pyspark/streaming/dstream.py", line 236, in func
    return old_func(rdd)  # type: ignore[call-arg, arg-type]
  File "/tmp/ipykernel_49918/2231400638.py", line 6, in <lambda>
    .foreachRDD(lambda rdd: rdd.toDF().sort(desc('count'))
  File "/home/juan/2023/Spark-Tutorials/.venv/lib/python3.10/site-packages/pyspark/sql/session.py", line 115, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/home/juan/2023/Spark-Tutorials/.venv/lib/python3.10/site-packages/pyspark/sql/session.py", line 1276, in createDataFrame
    return self._create_dataframe(
 

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `tweets` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 23;
'Project ['tag, 'count]
+- 'UnresolvedRelation [tweets], [], false


23/09/17 17:06:40 ERROR JobScheduler: Error running job streaming job 1694988400000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/home/juan/2023/Spark-Tutorials/.venv/lib/python3.10/site-packages/pyspark/streaming/util.py", line 71, in call
    r = self.func(t, *rdds)
  File "/home/juan/2023/Spark-Tutorials/.venv/lib/python3.10/site-packages/pyspark/streaming/dstream.py", line 236, in func
    return old_func(rdd)  # type: ignore[call-arg, arg-type]
  File "/tmp/ipykernel_49918/2231400638.py", line 6, in <lambda>
    .foreachRDD(lambda rdd: rdd.toDF().sort(desc('count'))
  File "/home/juan/2023/Spark-Tutorials/.venv/lib/python3.10/site-packages/pyspark/sql/session.py", line 115, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/home/juan/2023/Spark-Tutorials/.venv/lib/python3.10/site-packages/pyspark/sql/session.py", line 1276, in createDataFrame
    return self._create_dataframe(
 

In [ ]:
ssc.stop()